# Graph Analytics

This notebook has been created to test graph analytics capabilities of GraphFrame on a sample dataset.

In [1]:
# Import libraries 
import graphframe as GF
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import col

In [2]:
''' 
Function: Create spark context
Parameters: app_name, executor_memory, no_of_executors
Returns: Spark SQLContext
'''
def createContext(app_name="SocialGraph", executor_memory="2g", no_of_executors="4"):
    sparkConf = (SparkConf().setMaster("local").setAppName(app_name).set("spark.executor.memory", executor_memory).set("spark.executor.instances", no_of_executors))
    sparkContext = SparkContext(conf=sparkConf)
    sql_context = SQLContext(sparkContext)
    return sql_context

In [3]:
'''
Function: Create Spark dataframe from csv file
Parameters: file_path,sql_context
Returns: Pyspark Dataframe
'''
def loadData(file_path,sql_context):
    df = sql_context.read.format('com.databricks.spark.csv').options(header='true').load(file_path)
    return df

In [4]:
'''
Function: Provide stats for a Pyspark Dataframe
Parameters: Pyspark Dataframe
Returns: N/A
'''
def getStats(df):
    print("\nRow count: %d\n\nColumn Count: %d\n\nColumn headers: %s\n\nSample Data:\n" %(df.count(),len(df.columns),df.columns))
    df.show(5)  

In [5]:
'''
Function: Create vertices dataframe
Parameters: Pyspark Dataframe
Returns: Pyspark dataframe depciting vertices
'''
def createVertices(df):
    print("Creating Vertices DataFrame..")
    # Select user_Id [vertices] column from Spark dataframe
    df_users = df.select(['user_id'])
    df_users = df_users.selectExpr("user_id as id") 
    # Remove duplicate user_id entries and create vertices dataframe
    vertices = df_users.drop_duplicates()
    print("Vertices DataFrame creation complete.")
    return vertices

In [6]:
'''
Function: Create edges dataframe
Parameters: Pyspark Dataframe
Returns: Pyspark dataframe depciting edges
'''
def createEdges(df):
    # Create the edges dataframe
    print("Creating Edges DataFrame..")
    edges = df.select(col('user_id').alias('src'),col('group_id')).join(df.select(col('user_id').alias('dst'),col('group_id')), on=['group_id'], how='outer')
    edges = edges.select(col('src'),col('dst'),col('group_id')).filter(edges.src != edges.dst)
    print("Edges DataFrame creation complete.")
    return edges

In [7]:
'''
Function: Create graph
Parameters: Pyspark Dataframe - vertices, edges
Returns: GraphFrame
'''
def createGraph(vertices, edges):
    print("Creating graph..")
    # Generate the graph
    graph = GF.GraphFrame(vertices, edges)
    print("Graph creation complete.")
    return graph

In [8]:
'''
Function: Save graph to file
Parameters: GraphFrame
Returns: N/A
'''
def saveGraph(graph):
    # Save the graph to a file
    print("Saving graph to file..")
    graph.vertices.write.parquet('store/gv.parquet')
    graph.edges.write.parquet('store/ge.parquet')
    print("Graph has been saved successfully.")

In [9]:
'''
Function: Load graph from file
Parameters: N/A
Returns: GraphFrame
'''
def loadGraph(graph):
    # Load the graph from file
    print("\nLoading graph data..")
    vertices = spark.read.parquet('store/gv.parquet')
    edges = spark.read.parquet('store/ge.parquet')
    print("\Generating graph..")
    graph = GF.GraphFrame(vertices, edges)
    print("\nGraph load complete.")
    return graph

In [10]:
'''
Function: Obtain the first connects of a given vertex
Parameters: GraphFrame, vertex label
Returns: GraphFrame of connected vertices and their edges
'''
def first_connects(graph, vertex):
    first_connect_motifs = graph.find("(v1)-[e]->(v2)").filter("v1.id == '"+vertex+"'")
    return first_connect_motifs.select("v2.id","e")

In [11]:
'''
Function: Delete dataframe to free memory
Parameters: List of DataFrames
Returns: N/A
'''
def cleanUp(df_list):
    for df in df_list:
        del df
    print("\nDataFrame clean up complete.")

In [12]:
if __name__ == "__main__":
    
    # Create Spark context
    sql_context = createContext(app_name="SocialGraph", executor_memory="2g", no_of_executors="4")
    
    # Load group to member data into a Pyspark Dataframe
    df_group_members = loadData('data/group_members.csv',sql_context)
    
    # Load group to channel data into a Pyspark Dataframe
    df_group_channels = loadData('data/group_channel.csv',sql_context)
    
    # Get stats on group members and group channels dataframes
    getStats(df_group_members)
    getStats(df_group_channels)
    
    # Create vertices dataframe
    vertices = createVertices(df_group_members)
    
    # Create edges dataframe
    edges = createEdges(df_group_members)
    
    # Get stats on vertices and edges dataframes
    getStats(vertices)
    getStats(edges)
    
    # Create Graph
    duta_graph = createGraph(vertices,edges)
    
    # Save Graph to file 
    #saveGraph(duta_graph)
    
    # Load graph from file
    #duta_graph = loadGraph()
    
    
    


Row count: 20917620

Column Count: 6

Column headers: ['create_time', 'update_time', 'group_id', 'user_id', 'is_admin', 'added_by']

Sample Data:

+--------------------+--------------------+--------------------+--------------------+--------+--------+
|         create_time|         update_time|            group_id|             user_id|is_admin|added_by|
+--------------------+--------------------+--------------------+--------------------+--------+--------+
|2016-06-25 12:12:...|2016-06-25 12:12:...|c8a5ca5947cc7d925...|fcaa201f972dfc428...|       f|    null|
|2018-03-26 18:30:...|2018-03-26 18:30:...|d2c8410bb78af4615...|26cf4a8044866c1bc...|       f|    null|
|2017-12-18 10:23:...|2017-12-18 10:23:...|2c07f080d9f041295...|570a2ddfdccc2b775...|       f|    null|
|2018-02-08 16:58:...|2018-02-08 16:58:...|1430c9033779d7585...|df8d3021822ef6fed...|       f|    null|
|2018-03-26 18:30:...|2018-03-26 18:30:...|d2c8410bb78af4615...|777862a1db383bcd7...|       f|    null|
+-------------------